In [2]:
DIR = "./results"
import os 
import dill
from utils.state import State
import torch
from utils.environment import ENV
RESULTS_DIR = "./results"


def eval(state: State, device):
    state.model.eval()
    total = 0
    correct1 = 0
    correct5 = 0
    with torch.no_grad():
        for data, target in state.test_loader:
            data, target = data.to(device), target.to(device)
            output = state.model(data)
            total += state.loss(output, target).item() * data.size(0)
            _, pred = output.topk(5, dim=1)
            correct = pred.eq(target.view(-1, 1).expand_as(pred))
            correct1 += correct[:,:1].sum().item()
            correct5 += correct[:,:5].sum().item()

    test_loss = total / len(state.test_loader.dataset)
    accuracy = 100. * correct1 / len(state.test_loader.dataset)
    state.set_best_accuracy(accuracy)
    return accuracy

dataloader = None
for file in os.listdir(DIR):
    print(file)
    with open(f"{DIR}/{file}", "rb") as f:
        result = dill.load(f)
        s = State()
        if dataloader is None:
            s.load(result["file"], load_data=True)
            dataloader = s.test_loader
        else:
            s.load(result["file"], load_data=False)
            s.test_loader = dataloader
        result["accuracy"] = eval(s, ENV.device)
        with open(f"{RESULTS_DIR}/{result['file']}_dict.dill", "wb") as f:
            dill.dump(result, f)


standard_o1_snip_0.5_0_20220524051008236057_dict.dill
Loading mnist dataset.
standard_i1_hyperscorer_0.01_0_20220523225414598572_dict.dill
standard_i1_mag_0.5_0_20220524060239541348_dict.dill
grasp_o10_hyperscorer_v2_0.005_3_20220524030246359591_dict.dill
grasp_o10_hyperscorer_v2_0.5_0_20220524035352893183_dict.dill
snip_o10_hyperscorer_v2_0.2_3_20220524024620198410_dict.dill
snip_o10_hyperscorer_v2_0.007_3_20220524020446062600_dict.dill
grasp_o10_snip_0.005_3_20220524030344300785_dict.dill
standard_o10_snip_0.005_0_20220523225741915467_dict.dill
standard_i1_synflow_0.015_0_20220524014314452671_dict.dill
standard_o1_hyperscorer_0.005_3_20220523224829410569_dict.dill
standard_i10_synflow_0.005_6_20220523192358273470_dict.dill
standard_i1_rand_0.2_6_20220524041608930773_dict.dill
synflow_o10_mag_0.5_0_20220524014604464881_dict.dill
standard_o1_mag_0.5_0_20220524060154347627_dict.dill
grasp_o10_hyperscorer_v2_0.2_3_20220524035058017534_dict.dill
standard_o10_grasp_0.005_6_2022052319461097

In [7]:
DIR = "./results"
import os 
import dill

for file in os.listdir(DIR):
    with open(f"{DIR}/{file}", "rb") as f:
        result = dill.load(f)
        repeat_nr = file.rsplit("_", 3)[1]
        result["repeat_nr"] = repeat_nr
        with open(f"{DIR}/{result['file']}_dict.dill", "wb") as f:
            dill.dump(result, f)

In [1]:
FROM_DIR = "./results"
TO_DIR = "./results/Archive"

import os 
import dill

for file in os.listdir(FROM_DIR):
    if not os.path.isdir(f"{FROM_DIR}/{file}"):
        split = file.split("_")
        if split[2] in ['PyTorchModel', 'RandomForest', 'SGD', 'MLP', 'DecisionTree', 'AdaBoost', 'GradientBoosting']:# and int(split[5][:8]) < 20220603:
            print(file)
            os.rename(f"{FROM_DIR}/{file}", f"{TO_DIR}/{file}")

standard_o10_PyTorchModel_0.004_3_20220603191030957849_dict.dill
standard_o10_DecisionTree_0.004_0_20220603191501836115_dict.dill
standard_o10_PyTorchModel_0.005_3_20220603191732813298_dict.dill
standard_o10_DecisionTree_0.015_3_20220603194145245714_dict.dill
standard_o10_DecisionTree_0.007_3_20220603192911640852_dict.dill
standard_o10_PyTorchModel_0.01_3_20220603193038810910_dict.dill
standard_o10_PyTorchModel_0.007_0_20220603192548048244_dict.dill
standard_o10_RandomForest_0.2_3_20220603194343811917_dict.dill
standard_o10_DecisionTree_0.01_3_20220603193650645433_dict.dill
standard_o10_PyTorchModel_0.004_0_20220603191030900676_dict.dill
standard_o10_PyTorchModel_0.01_0_20220603193055850668_dict.dill
standard_o10_RandomForest_0.005_0_20220603192136263150_dict.dill
standard_o10_RandomForest_0.007_0_20220603192709510661_dict.dill
standard_o10_RandomForest_0.004_0_20220603191253646294_dict.dill
standard_o10_PyTorchModel_0.004_6_20220603191030918160_dict.dill
standard_o10_RandomForest_0.00

In [29]:
import os
import shutil 

all_results = [file.rsplit("_",1)[0] for file in os.listdir("./results")]

#clean up runs:
kept = 0
deleted = 0
for run in os.listdir("./runs"):
    if not run in all_results:
        # shutil.rmtree("./runs/"+run)
        deleted += 1
    else:
        kept += 1

kept, deleted

(7100, 0)

In [30]:
#clean up saves:
kept = 0
deleted = 0
for save in os.listdir("./saves"):
    if not os.path.isdir("./saves/"+save):
        if not save.rsplit(".", 1)[0] in all_results:
            # os.remove("./saves/"+save)
            deleted += 1
        else:
            kept += 1

kept, deleted

(7100, 0)

In [31]:
from analysis.results_utils import load_csv

df = load_csv()

all_init_ids = df["base_model_id"].to_list()


kept = 0
deleted = 0
for init_id in os.listdir("./saves/base_models"):
    if not init_id.rsplit(".", 1)[0] in all_init_ids:
        # os.remove("./saves/base_models/"+init_id)
        deleted += 1
    else:
        kept += 1

kept, deleted


(1630, 0)